# GAIA DR2 cross-match

In [1]:
# Authors: Lauren Anderson, Megan Bedell, Chiara Mingarelli
# Please cite Mingarelli, Anderson, Bedell and Spergel (in prep)

In [1]:
# %load query.py
import numpy as np
from astropy import units as u
from astropy.io import ascii
from astroquery.xmatch import XMatch
import requests

In [2]:
# %load query.py
import numpy as np
import glob

from astropy import units as u
from astropy.io import ascii
from astroquery.xmatch import XMatch
from astropy.table import Table, vstack
from astropy.coordinates import SkyCoord, ICRS, Galactic, Angle
from astropy.time import Time

## Upload your files with the info you'd like to cross-match with GAIA DR2

In [3]:
files = glob.glob('../IPTA_DR1/*/*.par') # We are interested in pulsars, these files contain their parameters

## Here we parse the pulsar .par files to get the known pulsar positions and proper motions. This is specific to the pulsar data format used by the International Pulsar Timing Array.

In [15]:
RAs = []
DECs = []
Names = []
pmra = []
pmdec = []
pepoch = []
posepoch = []
dmepoch = []
dm = []
coords = []

for file in files:
    d = {}
    with open(file) as f:
        for line in f:
            key = line.split()[0]
            value = line.split()[1]
            d[key] = value
            
    try:
        ra = Angle(d['RAJ'], u.hourangle).to(u.deg)
        dec = Angle(d['DECJ'], u.degree)
        coord = SkyCoord(ra=ra, dec=dec, frame='icrs',
                     pm_ra_cosdec = float(d['PMRA'])*u.mas/u.yr, # bug updated, thanks Paul Ray
                     pm_dec=float(d['PMDEC'])*u.mas/u.yr,
                     obstime=Time(float(d['POSEPOCH']), format='mjd'),
                     distance=1. * u.kpc) # HACK
        coords.append(coord)
        RAs.append(ra.value)
        #print(d['PSRJ'], d['DECJ'])
        DECs.append(dec.value)
        Names.append(d['PSRJ'])
        pmra.append(coord.pm_ra_cosdec.value)
        pmdec.append(coord.pm_dec.value)
        pepoch.append(d['PEPOCH'])
        posepoch.append(d['POSEPOCH'])
        dmepoch.append(d['DMEPOCH'])
        dm.append(d['DM'])
        
    except KeyError:
        lon = float(d['ELONG'])*u.deg
        lat = float(d['ELAT'])*u.deg
        coord = SkyCoord(l=lon, b=lat, frame='galactic',
                         pm_l_cosb = float(d['PMELONG'])*u.mas/u.yr, 
                         pm_b=float(d['PMELAT'])*u.mas/u.yr,
                         obstime=Time(float(d['POSEPOCH']), format='mjd'),
                         distance=1. * u.kpc) # HACKITY HACK HACK
        icrscoord = coord.transform_to(ICRS)
        coords.append(icrscoord)
        #print(icrscoord)
        #print(d['PSRJ'])
        RAs.append(icrscoord.ra.to(u.deg).value)
        DECs.append(icrscoord.dec.to(u.deg).value)
        Names.append(d['PSRJ'])
        pmra.append(icrscoord.pm_ra_cosdec.value)
        pmdec.append(icrscoord.pm_dec.value)
        pepoch.append(d['PEPOCH'])
        posepoch.append(d['POSEPOCH'])
        dmepoch.append(d['DMEPOCH'])
        dm.append(d['DM'])        
        #coord = SkyCoord(d[''])
        
        continue
        
t = Table([Names, RAs, DECs, pmra, pmdec, pepoch, posepoch, dmepoch, dm], names=['names', 'ra', 'dec', 'pmra', 'pmdec', 'dm', 'pepoch', 'posepoch', 'dmepoch'])        
#tmin = Table([RAs, DECs], names=['ra', 'dec'])


In [16]:
filename = 'ipta_data_nopm'
t.write(filename+'.csv', format='csv', overwrite=True)
t.write(filename+'.vot', format='votable', overwrite=True)

## Here we propagate the pulsar proper motion to update them to the GAIA DR2 reference epoch... or rather J2000 since this seems to be what XMatch uses?

In [17]:
j2000_time = Time('J2000.000')

In [18]:
t['ra'] = [c.apply_space_motion(new_obstime=gaia_time).ra.value for c in coords]
t['dec'] = [c.apply_space_motion(new_obstime=gaia_time).dec.value for c in coords]

In [19]:
filename = 'ipta_data_j2000'
t.write(filename+'.csv', format='csv', overwrite=True)
t.write(filename+'.vot', format='votable', overwrite=True)

# We tell the x-match server that we want to search GAIA DR2 for objcts at the RA and DEC we specifiy in our file, within a 3 arsecond search radius.

In [20]:
filename = 'ipta_data_j2000'
r = requests.post(
'http://cdsxmatch.u-strasbg.fr/xmatch/api/v1/sync',
data={'request': 'xmatch', 'distMaxArcsec': 3.0, 'RESPONSEFORMAT': 'csv',
'cat2': 'vizier:I/345/gaia2', 'colRA1': 'ra', 'colDec1': 'dec'},
files={'cat1': open(filename+'.csv', 'r')})

In [21]:
h = open('xmatch_j2000.csv', 'w')
h.write(r.text)
h.close()

And for comparison, here it is without PM propagation:

In [12]:
filename = 'ipta_data_nopm'
r = requests.post(
'http://cdsxmatch.u-strasbg.fr/xmatch/api/v1/sync',
data={'request': 'xmatch', 'distMaxArcsec': 3.0, 'RESPONSEFORMAT': 'csv',
'cat2': 'vizier:I/345/gaia2', 'colRA1': 'ra', 'colDec1': 'dec'},
files={'cat1': open(filename+'.csv', 'r')})

In [13]:
h = open('xmatch_nopm.csv', 'w')
h.write(r.text)
h.close()